In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers

     |████████████████████████████████| 4.2 MB 8.4 MB/s 
     |████████████████████████████████| 596 kB 65.9 MB/s 
     |████████████████████████████████| 84 kB 3.7 MB/s 
     |████████████████████████████████| 6.6 MB 44.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [ ]:
train = pd.read_csv("/content/drive/MyDrive/Datasets/Combine/Combined.csv")
train.head()

,id,text,class_label
0,2760,আফতাব আপনি ভালো আছেন?,neutral
1,955,মমিনুল হক আর মুশফিক দুজনেই নিজেদের উইকেট জঘন্য...,positive
2,43,মিরপুর এর পিচ এ যে কেও বোলিং করতে পারে,neutral
3,2211,মুদ্রার ঐ পিঠ দেখা হয়ে গেল বাংলাদেশের.দেশের মা...,negative
4,2490,শুভকামনা রাজ্জাক ভাই।,negative


In [ ]:
labels = list(set(train['class_label']))
print(labels)

['Neutral', 'BN_POS', 'BN_NEG', 'Positive', 'negative', 'neutral', 'BN_NEU', 'Negative', 'positive']


In [ ]:
train['class_label'] = train['class_label'].replace({'Negative':0, 'Positive':1, 'Neutral':2, 'negative':0, 'BN_POS':1, 'neutral':2,'BN_NEG':0, 'positive':1, 'BN_NEU':2})

In [ ]:
labels = list(set(train['class_label']))
print(labels)
del train['id']
train.head()

[0, 1, 2]


,text,class_label
0,আফতাব আপনি ভালো আছেন?,2
1,মমিনুল হক আর মুশফিক দুজনেই নিজেদের উইকেট জঘন্য...,1
2,মিরপুর এর পিচ এ যে কেও বোলিং করতে পারে,2
3,মুদ্রার ঐ পিঠ দেখা হয়ে গেল বাংলাদেশের.দেশের মা...,0
4,শুভকামনা রাজ্জাক ভাই।,0


In [ ]:
train_texts = train['text'].values.tolist()
train_labels = train['class_label'].values.tolist()

In [ ]:
from sklearn.model_selection import train_test_split
train_texts, val_texts, train_labels, val_labels =train_test_split(train_texts, train_labels,
                 test_size=.2, random_state=42, stratify=train_labels)

In [ ]:
import torch
from torch.utils.data import Dataset
from transformers import AutoTokenizer,AutoModelForSequenceClassification, AlbertTokenizer
from transformers import Trainer,TrainingArguments

In [ ]:
# tokenizer = AutoTokenizer.from_pretrained('/content/drive/MyDrive/Models/CSE498',num_labels=3)
tokenizer = AutoTokenizer.from_pretrained('/content/drive/MyDrive/Models/pet_bnbert/p0-i0',num_labels=3)
# tokenizer = AutoTokenizer.from_pretrained('/content/drive/MyDrive/Models/pruned_bnbert',num_labels=3)

In [ ]:
train_encodings = tokenizer(train_texts, truncation=True, max_length=512,padding=True,return_tensors = 'pt')
val_encodings = tokenizer(val_texts, truncation=True, max_length=512,padding=True,return_tensors = 'pt')

In [ ]:
class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)
## Test Dataset
class SentimentTestDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        return item
    def __len__(self):
        return len(self.encodings)

In [ ]:
train_dataset = SentimentDataset(train_encodings, train_labels)
val_dataset = SentimentDataset(val_encodings, val_labels)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, average_precision_score
def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    y_score = pred
    y_true = labels

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred, average='weighted')
    precision = precision_score(y_true, y_score, average='weighted')
    f1 = f1_score(labels, pred, average='weighted')

    return {"accuracy": accuracy,"f1_score":f1, "recall": recall, "precision": precision}

In [ ]:
training_args = TrainingArguments(
    output_dir='./res',          # output directory
    evaluation_strategy="steps",
    num_train_epochs=30,              # total number of training epochs
    per_device_train_batch_size=32,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs4',            # directory for storing logs
    #logging_steps=10,
    load_best_model_at_end=True,
)

In [ ]:
# model = AutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/Models/CSE498', num_labels=3)
model = AutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/Models/pet_bnbert/p0-i0', num_labels=3)
# model = AutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/Models/pruned_bnbert', num_labels=3)

# for name, param in model.named_parameters():
#      if name.startswith("bert.encoder.layer.1"): # choose whatever you like here
#         param.requires_grad = False

trainer = Trainer(
    model=model,# the instantiated 🤗 Transformers model to be trained
    args=training_args, # training arguments, defined above
    train_dataset=train_dataset,# training dataset
    eval_dataset=val_dataset , # evaluation dataset
    compute_metrics=compute_metrics,
)


In [ ]:
trainer.train()

***** Running training *****
  Num examples = 12970
  Num Epochs = 30
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 12180


Step,Training Loss,Validation Loss,Accuracy,F1 Score,Recall,Precision
500,1.095200,0.846526,0.633981,0.633981,0.633981,0.649582
1000,0.730600,0.738985,0.705211,0.705211,0.705211,0.722540
1500,0.552400,0.961787,0.613321,0.613321,0.613321,0.727827
2000,0.414500,0.783187,0.718779,0.718779,0.718779,0.729863
2500,0.295400,0.966417,0.713537,0.713537,0.713537,0.713566
3000,0.222800,1.054813,0.712612,0.712612,0.712612,0.710485
3500,0.164900,1.206341,0.699661,0.699661,0.699661,0.709289
4000,0.153100,1.220630,0.707370,0.707370,0.707370,0.709096
4500,0.114800,1.450639,0.701203,0.701203,0.701203,0.707410
5000,0.100800,1.568637,0.685785,0.685785,0.685785,0.682561


***** Running Evaluation *****
  Num examples = 3243
  Batch size = 64
Saving model checkpoint to ./res/checkpoint-500
Configuration saved in ./res/checkpoint-500/config.json
Model weights saved in ./res/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 3243
  Batch size = 64
Saving model checkpoint to ./res/checkpoint-1000
Configuration saved in ./res/checkpoint-1000/config.json
Model weights saved in ./res/checkpoint-1000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 3243
  Batch size = 64
Saving model checkpoint to ./res/checkpoint-1500
Configuration saved in ./res/checkpoint-1500/config.json
Model weights saved in ./res/checkpoint-1500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 3243
  Batch size = 64
Saving model checkpoint to ./res/checkpoint-2000
Configuration saved in ./res/checkpoint-2000/config.json
Model weights saved in ./res/checkpoint-2000/pytorch_model.bin
***** Running Evaluation *****
  Num exampl

TrainOutput(global_step=12180, training_loss=0.18449392289363692, metrics={'train_runtime': 4131.3286, 'train_samples_per_second': 94.183, 'train_steps_per_second': 2.948, 'total_flos': 5723115388416000.0, 'train_loss': 0.18449392289363692, 'epoch': 30.0})

In [ ]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 3243
  Batch size = 64


{'epoch': 30.0,
 'eval_accuracy': 0.7052112241751465,
 'eval_f1_score': 0.7052112241751465,
 'eval_loss': 0.7389847040176392,
 'eval_precision': 0.7225399105431426,
 'eval_recall': 0.7052112241751465,
 'eval_runtime': 12.7196,
 'eval_samples_per_second': 254.961,
 'eval_steps_per_second': 4.01}